In [2]:
import json
import numpy as np
import cv2
import mediapipe as mp
import os

In [6]:
GESTURE_FILE = "gestures.json"

if not os.path.exists(GESTURE_FILE):
    print("No gestures found.")
    exit()

with open(GESTURE_FILE, "r") as f:
    gesture_dict = json.load(f)
    gesture_dict = {
        k: {stage: np.array(v, dtype=np.float32) for stage, v in frames.items()}
        for k, frames in gesture_dict.items()
    }

# Initialize Mediapipe for drawing
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Define hand connections based on Mediapipe's model
HAND_CONNECTIONS = mp_hands.HAND_CONNECTIONS

# Function to denormalize landmarks for visualization
def denormalize_landmarks(landmarks, width=320, height=240):
    """Denormalizes landmarks to fit within the screen dimensions."""
    scale = min(width, height) // 2  # Assume a scaling factor
    center_x, center_y = width // 2, height // 2
    denormalized = landmarks * scale + np.array([center_x, center_y, 0])
    return denormalized

# Create an OpenCV window
cv2.namedWindow("Hand Gesture", cv2.WINDOW_NORMAL)

for gesture_name, keyframes in gesture_dict.items():
    for stage, keypoints in keyframes.items():
        # Create a blank image
        img = np.zeros((240, 320, 3), dtype=np.uint8)

        # Denormalize keypoints
        denorm_landmarks = denormalize_landmarks(keypoints)

        # Convert to Mediapipe format
        landmarks_list = []
        for i, (x, y, z) in enumerate(denorm_landmarks):
            landmarks_list.append(mp.solutions.hands.HandLandmark(i))

        # Draw landmarks and connections
        for i, (x, y, _) in enumerate(denorm_landmarks):
            cv2.circle(img, (int(x), int(y)), 5, (0, 255, 0), -1)  # Draw points

        for connection in HAND_CONNECTIONS:
            start_idx, end_idx = connection
            if start_idx < len(denorm_landmarks) and end_idx < len(denorm_landmarks):
                start_point = tuple(map(int, denorm_landmarks[start_idx][:2]))
                end_point = tuple(map(int, denorm_landmarks[end_idx][:2]))
                cv2.line(img, start_point, end_point, (255, 255, 255), 2)  # Draw connections

        # Display gesture name and stage
        cv2.putText(img, f"{gesture_name} - {stage}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Show the gesture in a single window
        cv2.imshow("Hand Gesture", img)
        cv2.waitKey(2000)  # Wait 2 seconds before moving to the next gesture

cv2.destroyAllWindows()

KeyboardInterrupt: 